In [1]:

# Step 1: Define Sample Documents
documents = [
    {"section": "Employee Info", "content": "John's pay is processed on the 1st of every month."},
    {"section": "Employee Info", "content": "Mark is on a leave of absence until next Monday."},
    {"section": "Employee Info", "content": "Julie is a software engineer."},
    {"section": "Employee Info", "content": "Julie's pay is processed on the 1st of every month."},
    {"section": "Employee Info", "content": "Mark is a product manager."},
    {"section": "Employee Info", "content": "John is an AI architect and has salary of 500K USD."},
]

# Step 2: Get Content Texts
content_corpus = [doc["content"] for doc in documents]
content_corpus

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
doc_vectors = model.encode(content_corpus)

doc_vectors
print(doc_vectors.shape)


/Users/amiteshsinha/Training/10_2025_genai_lab/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(6, 384)


In [2]:
# Step 3: User Query and Semantic Matching
import numpy as np

query = "Tell me about John's role."
query_vec = model.encode([query])[0]
# query_vec


similarities = model.similarity(query_vec, doc_vectors)

# Ensure it's a 1D numpy array
similarities = np.asarray(similarities).squeeze()

# Now get top 3
top_3_indices = np.argsort(similarities)[::-1][:3]
top_scores = similarities[top_3_indices]
top_scores

top_docs = [documents[i]['content'] for i in top_3_indices]

top_docs
context = "\n---\n".join(top_docs)
top_docs, context

(["John's pay is processed on the 1st of every month.",
  'John is an AI architect and has salary of 500K USD.',
  'Mark is a product manager.'],
 "John's pay is processed on the 1st of every month.\n---\nJohn is an AI architect and has salary of 500K USD.\n---\nMark is a product manager.")

In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True, dotenv_path="../.env")
my_api_key = os.getenv("OPEN_AI_API_KEY")

my_client = OpenAI(api_key=my_api_key)
# my_client

def ask_question_open_ai(prompt, context=''):
    """Call the LLM with the provided prompt and context.

    IMPORTANT: use the passed-in prompt (not a global variable) so each
    evaluation example can be answered correctly.
    """
    llm_response = my_client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": '''
             You are an assistant who answers only based on the given context.
             '''},
            {"role": "user", "content": f"Context: {context}\n\nUser Question: {prompt}"}
        ]

    )
    return llm_response.choices[0].message.content


In [15]:
print (query)
response = ask_question_open_ai(query, context)

response

Tell me about John's role.


'John’s role is AI architect.'

#### Create a LangSmith Dataset

In [4]:
from langsmith import Client

client = Client()

dataset_name = "2025Dec-Employee-Info-QA-Dataset-3"
dataset = client.create_dataset(dataset_name=dataset_name)

examples = [
    {"input": "When is John's pay processed?", "output": "John's pay is processed on the 1st of every month."},
    {"input": "What is Julie's job title?", "output": "Julie is a software engineer."},
    {"input": "What is John's salary?", "output": "John has a salary of 500K USD."},
    {"input": "What is Mark's current work status?", "output": "Mark is on a leave of absence until next Monday."},
]

for ex in examples:
    client.create_example(inputs={"question": ex["input"]}, outputs={"answer": ex["output"]}, dataset_id=dataset.id)

print(f" Dataset '{dataset_name}' created with {len(examples)} examples.")


 Dataset '2025Dec-Employee-Info-QA-Dataset-3' created with 4 examples.


##### Run LangSmith Evaluation

In [5]:
# %pip install --upgrade --no-cache-dir langsmith

import langsmith, langchain
print(langsmith.__version__)
print(langchain.__version__)


0.4.42
1.1.3


In [19]:
# !pip index versions langsmith

In [7]:
import os
import json
from openai import OpenAI
from langsmith.evaluation import RunEvaluator

class SimpleCorrectness(RunEvaluator):
    """LLM-as-a-judge correctness evaluator (version-safe)."""

    def evaluate_run(self, run, example, **kwargs):
        ref = example.outputs.get("answer", "").strip()

        pred = (
            run.outputs.get("answer")
            or run.outputs.get("output_text")
            or run.outputs.get("result")
            or ""
        ).strip()

        question = example.inputs.get("question", "")

        if not ref or not pred:
            return {
                "key": "correctness",
                "score": 0.0,
                "commentary": "Missing reference or prediction",
            }

        try:
            score, reason = self.llm_as_a_judge(ref, pred, question)
        except Exception as e:
            return {
                "key": "correctness",
                "score": 0.0,
                "commentary": f"Evaluator error: {e}",
            }

        return {
            "key": "correctness",
            "score": score,
            "commentary": reason,
        }

    def llm_as_a_judge(self, reference: str, prediction: str, question: str = ""):
        client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0,
            messages=[
                {
                    "role": "system",
                    "content": "You are a semantic correctness evaluator."
                },
                {
                    "role": "user",
                    "content": f"""
Question: {question}
Reference answer: {reference}
Model prediction: {prediction}

Return JSON only:
{{"score": <number between 0 and 1>, "reason": "<short explanation>"}}
"""
                }
            ],
        )

        content = response.choices[0].message.content
        data = json.loads(content)

        score = float(data["score"])
        reason = data["reason"]

        return max(0.0, min(1.0, score)), reason


In [9]:
# from langchain_openai import ChatOpenAI
from langsmith import traceable # Need to enable tracing on LangSmith

# Define your target function that performs retrieval per-question
@traceable 
def ask_question(inputs):
    question = inputs["question"]
    # compute embedding for the question
    query_vec = model.encode([question])[0]

    # compute cosine similarities between query and doc_vectors
    import numpy as np
    q_norm = np.linalg.norm(query_vec) + 1e-8
    doc_norms = np.linalg.norm(doc_vectors, axis=1) + 1e-8
    sims = np.dot(doc_vectors, query_vec) / (doc_norms * q_norm)

    # pick top-3 supporting docs and build context
    top_3_indices = np.argsort(sims)[::-1][:3]
    top_docs = [content_corpus[i] for i in top_3_indices]
    context = "\n---\n".join(top_docs)

    # call LLM with question and its retrieved context
    answer = ask_question_open_ai(question, context)
    return {"answer": answer}


In [10]:
import os
import json
from openai import OpenAI
from langsmith.evaluation import RunEvaluator

class SimpleCorrectness(RunEvaluator):
    """LLM-as-a-judge correctness evaluator (version-safe)."""

    def evaluate_run(self, run, example, **kwargs):
        ref = example.outputs.get("answer", "").strip()

        pred = (
            run.outputs.get("answer")
            or run.outputs.get("output_text")
            or run.outputs.get("result")
            or ""
        ).strip()

        question = example.inputs.get("question", "")

        if not ref or not pred:
            return {
                "key": "correctness",
                "score": 0.0,
                "commentary": "Missing reference or prediction",
            }

        try:
            score, reason = self.llm_as_a_judge(ref, pred, question)
        except Exception as e:
            return {
                "key": "correctness",
                "score": 0.0,
                "commentary": f"Evaluator error: {e}",
            }

        return {
            "key": "correctness",
            "score": score,
            "commentary": reason,
        }

    def llm_as_a_judge(self, reference: str, prediction: str, question: str = ""):
        client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0,
            messages=[
                {
                    "role": "system",
                    "content": "You are a semantic correctness evaluator."
                },
                {
                    "role": "user",
                    "content": f"""
Question: {question}
Reference answer: {reference}
Model prediction: {prediction}

Return JSON only:
{{"score": <number between 0 and 1>, "reason": "<short explanation>"}}
"""
                }
            ],
        )

        content = response.choices[0].message.content
        data = json.loads(content)

        score = float(data["score"])
        reason = data["reason"]

        return max(0.0, min(1.0, score)), reason


In [11]:
from langsmith.evaluation import evaluate

simple_correctness = SimpleCorrectness()

results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[simple_correctness],
    experiment_prefix="langsmith_eval_test",
)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

View the evaluation results for experiment: 'langsmith_eval_test-a6a662cf' at:
https://smith.langchain.com/o/79133fd6-316c-4b99-b886-38847131d1e1/datasets/9a93e5d3-cdc8-4a90-aef2-9de7206c5a93/compare?selectedSessions=8e89748e-f376-46be-bf48-c5e3e5f60761




4it [00:19,  4.81s/it]


In [ ]:
for r in results:   # each r is a dict
    example = r["example"]
    eval_results = r["evaluation_results"]["results"]

    print(f"\n Question: {example.inputs['question']}")
    print(f"Expected: {example.outputs['answer']}")

    # Extract model output
    run = r["run"]
    if hasattr(run, "outputs") and "answer" in run.outputs:
        print(f"Predicted: {run.outputs['answer']}")
    else:
        print("Predicted: (no output found)")

    # Print evaluator results
    for e in eval_results:
        print(f"Evaluator: {e.key}, Score: {e.score}, Explanation: {getattr(e, 'explanation', None)}")



Question: What is Mark's current work status?
Expected: Mark is on a leave of absence until next Monday.
Predicted: Mark is currently on a leave of absence until next Monday.


AttributeError: 'EvaluationResult' object has no attribute 'commentary'